### Package Installation

In [ ]:
!pip install Cython
!pip install nemo_toolkit['all']
!pip uninstall -y torchmetrics
!pip install torchmetrics==0.9.2

### Necessary Imports

In [2]:
import os
from pydub import AudioSegment
import librosa
import soundfile as sf

### Audio File Format Conversion

In [3]:
def convert_to_wav(file_path,dest_path, format):
  print(f'Converting {file_path} to wav format...')
  audio = AudioSegment.from_file(file_path, format=format)
  audio.export(dest_path, format="wav")
  print(f'Converted {dest_path} to wav format.')
  return dest_path

In [4]:
convert_to_wav('/content/sample audio 1.m4a', '/content/sample audio 1.wav','m4a')
convert_to_wav('/content/sample audio 2.m4a', '/content/sample audio 2.wav','m4a')

Converting /content/sample audio 1.m4a to wav format...
Converted /content/sample audio 1.wav to wav format.
Converting /content/sample audio 2.m4a to wav format...
Converted /content/sample audio 2.wav to wav format.


'/content/sample audio 2.wav'

### Audio Resampling for BanglaConformer

In [5]:
def resample_audio(input_filepath, output_filepath, target_sr=16000):
    if not os.path.exists(output_filepath):
        # Load audio (automatically resampled to target_sr, default is 22050)
        audio, sr = librosa.load(input_filepath, sr=target_sr, mono=True)
        sf.write(output_filepath, audio, target_sr)

input_filepath = "/content/sample audio 2.wav"
output_filepath = "/content/sample_audio_2_resampled.wav"

resample_audio(input_filepath, output_filepath)


In [6]:
input_filepath = "/content/sample audio 1.wav"
output_filepath = "/content/sample_audio_1_resampled.wav"

resample_audio(input_filepath, output_filepath)

### Transcribing Audio to Text (bengaliAI/BanglaConformer)

In [7]:
import nemo.collections.asr as nemo_asr
asr_model = nemo_asr.models.EncDecCTCModelBPE.from_pretrained("bengaliAI/BanglaConformer")

BanglaConformer.nemo:   0%|          | 0.00/510M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

[NeMo I 2024-03-11 05:17:46 mixins:172] Tokenizer AutoTokenizer initialized with 32000 tokens


[NeMo W 2024-03-11 05:17:57 modelPT:165] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath: /mnt/richb/imtiaz/ood_speech/df_filtered_train_decoded.json
    sample_rate: 16000
    batch_size: 8
    shuffle: true
    num_workers: 8
    pin_memory: true
    use_start_end_token: false
    trim_silence: false
    max_duration: 30
    min_duration: 0.1
    is_tarred: false
    tarred_audio_filepaths: null
    shuffle_n: 2048
    bucketing_strategy: synced_randomized
    bucketing_batch_size: null
    
[NeMo W 2024-03-11 05:17:57 modelPT:172] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_validation_data() method and provide a valid configuration file to setup the validation data loader(s). 
    Validation config : 
    manifest_filepath: /mnt/richb/imtiaz/ood_speech/df_filter

[NeMo I 2024-03-11 05:17:57 features:289] PADDING: 0
[NeMo I 2024-03-11 05:18:10 save_restore_connector:249] Model EncDecCTCModelBPE was successfully restored from /root/.cache/huggingface/hub/models--bengaliAI--BanglaConformer/snapshots/0b122bdf5555f834bb9ed84de6929cfabfc2ef97/BanglaConformer.nemo.


In [8]:
asr_model.transcribe(paths2audio_files=['/content/sample_audio_1_resampled.wav','/content/sample_audio_2_resampled.wav','/content/temp_audio.wav'])

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

['আচ্ছা আমি এখন বেঙ্গল দরজায় বাংলা নামে যে মডেল কাছে ঐ দিয়ে আমাদের এক কোন ট্রান্স করার চেষ্টা করব, দেখব যে সেই কতটা ভালোভাবে ট্রান্স করতে ভাবতে বা তার ডিগ্রি ডি করতে ট্রান্স করার সময় এটাই ।',
 'আমি এখন প্রথম আলো থেকে কয়েকটি শিরোনাম দেব বলছে বেঙ্গল এই কাজ করে, যেমন প্রথম হচ্ছে আকর্ষণ আকর্ষণীয় স্থানে পর্যটন কর্পোরেশনের হোটেল তবু কেন লোকসান বিভেদ দল বোঝাতে রঙ কর্মসূচি আওয়ামী অস্কার সেরা সিনেমা ও ।',
 'আমি হলাম রা আমি একটা ভয়েস ম্যান রেকর্ড করছি এটা আমি একটা নিউজ কমান্ড করব যদি লম্বা ফাইল হয় আমি দেখতে যাচ্ছি ডেটাাইড করতে পারে কিনা গুগল ট্রান্স ।']